In [ ]:
import os, glob
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib
import seaborn as sns

# Drawing initialization
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
mpl.rcParams['axes.xmargin'] = 0
sns.set(font='IPAexGothic')

In [ ]:
# Get the date and time use for aggregation
now = dt.datetime.now()
first_day_of_this_month = dt.datetime(now.year, now.month, 1)
first_day_of_this_month = dt.datetime(2020, 11, 1)
last_day_of_last_month = first_day_of_this_month + dt.timedelta(days=-1)
first_day_of_last_month = dt.datetime(last_day_of_last_month.year, last_day_of_last_month.month, 1)

# Header year and month
yyyy = first_day_of_last_month.strftime('%Y')
mm = first_day_of_last_month.strftime('%m')
header = '{}-{}'.format(yyyy, mm)
print(header)

# Mackerel Report

In [ ]:
target_dir = os.path.join(os.getcwd(), 'data')
yyyymm = first_day_of_last_month.strftime('%Y%m')
file_list = glob.glob(target_dir + '\\{}\\*'.format(yyyymm))

unit_list = []

# Preprocessing: Generate a processing unit list unit to create a graph
for file in file_list:

    file_name = os.path.basename(file).split('.')

    if 'filesystem' in file_name:
        unit = '{}.{}.{}'.format(file_name[0], file_name[1], file_name[2])
    else:
        unit = '{}.{}'.format(file_name[0], file_name[1])

    unit_list.append(unit)

unit_list = sorted(list(set(unit_list)))

# Main process: draw a graph for each unit
for unit in unit_list:

    fig = plt.figure(figsize=(30, 8))
    ax = plt.subplot()
    legend = []

    unit_parts = unit.split('.')
    if 'cpu' in unit_parts:
        print('{} - {}'.format(unit_parts[0], unit_parts[1].upper()))
    elif 'filesystem' in unit_parts:
        print('{} - FileSystem {}:'.format(unit_parts[0], unit_parts[2]))
    elif 'memory' in unit_parts:
        print('{} - {}'.format(unit_parts[0], unit_parts[1].upper()))
    
    for file in file_list:

        if unit in file:

            df = pd.read_csv(file)
            df['time'] = pd.to_datetime(df['time'].astype(int), unit='s')
            df = df.set_index(['time'])
            df = df.reindex(pd.date_range(first_day_of_last_month, first_day_of_this_month, freq="H"))
            df['value'] = df['value'].fillna(0)
            if 'filesystem' in unit or 'memory' in unit:
                df['value'] = df['value']/1024/1024/1024

            x = df.index
            y = df['value']
            ax.plot(x, y, label="%")

            file_name = os.path.basename(file).split('.')

            if 'filesystem' in file_name:
                metrics = file_name[3]
            else:
                metrics = file_name[2]
            
            legend.append(metrics)

    ax.set_title(unit)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
    plt.xticks(rotation=90)
    if 'cpu' in unit:
        plt.yticks(np.arange(0, 100 + 1, 10))
        plt.ylabel("percentage")
    else:
        plt.ylabel("GB")
    plt.legend(legend)
    plt.subplots_adjust(left=0.10, right=0.95, top=0.90, bottom=0.15, wspace=0.50, hspace=0.00)
    plt.show()
    plt.close('all')

    print('\n')

以上